# Prompt:
```
Generate a JSON object for a YouTube video or short reel with the following specifications:

Topic: [Your Topic Here in English]
Language: [Your Language Here]
Length: [Specify the desired length of the video; the content should be engaging and fit this duration.]
Please structure the JSON object with the following fields:

Title: (Create a catchy title that attracts views.)
Description: (Provide a brief and compelling description for the video.)
Story Narration:
Introduction: (Start with a friendly greeting, like "Hello!" or "Hi there!" based on the topic.)
Content: (Main body of narration; craft engaging and interesting content based on the specified length and topic, in the desired language.)
Conclusion: (Wrap up with a summary and a call-to-action encouraging viewers to subscribe, hit the bell icon, and stay tuned.)
Tags: (List keywords relevant to the topic for improved visibility.)
Narration Type: (Define the narration style, e.g., "Storytelling," "Informative," "Review.")
Category ID: (Set YouTube category ID, e.g., "24" for Entertainment.)
Privacy Status: (Choose privacy settings, e.g., "public.")
Thumbnail Text: (Generate attention-grabbing text for the thumbnail.)
Image Tags: (Provide tags or keywords for sourcing relevant images.)
Music Suggestions: (Suggest background music types or genres that fit the video's mood.)
Audio Language (for TTS): (Specify the text-to-speech language code, e.g., "en" for English, "te" for Telugu.)
Script Revision Note: (Add brief notes about any revisions made to the script.)
Upload Date and Time (Optional): (Schedule the upload timing if needed; default is immediate.)
Topic_Language: (Always in English; combine the topic and language, e.g., "Katrina Kaif Biography - Telugu".)
```

In [11]:
import os
import json
import requests
from moviepy.editor import *
from gtts import gTTS
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()

# Global access key for Unsplash
access_key = os.getenv('UNSPLASH_ACCESS_KEY')

def load_json_data(json_path):
    """Load data from a JSON file."""
    with open(json_path, 'r') as f:
        return json.load(f)

def prepare_variables(json_data):
    """Prepare variables from loaded JSON data."""
    topic_language = json_data['Topic_Language']
    title = json_data['Title']
    description = json_data['Description']
    story_narration = json_data['Story Narration']
    tags = json_data['Tags']
    narration_type = json_data['Narration Type']
    privacy_status = json_data['Privacy Status']
    thumbnail_text = json_data['Thumbnail Text']
    image_tags = json_data['Image Tags']
    music_suggestions = json_data['Music Suggestions']
    audio_language = json_data['Audio Language (for TTS)']
    script_revision_note = json_data['Script Revision Note']
    upload_date_time = json_data.get('Upload Date and Time', 'immediate')

    return topic_language, title, description, story_narration, tags, narration_type, privacy_status, thumbnail_text, image_tags, music_suggestions, audio_language, script_revision_note, upload_date_time

def prepare_file_names(topic_language, resources_path):
    """Prepare file names based on topic language."""
    video_file_name = f"{topic_language}.mp4"
    audio_file_name = f"{topic_language}.mp3"
    output_video_path = os.path.join(resources_path, "videos", video_file_name)
    
    return video_file_name, audio_file_name, output_video_path

def download_images(image_tags, resources_path, topic_language):
    """Download images if local resources are not available."""
    images_folder = os.path.join(resources_path, topic_language, "images")
    os.makedirs(images_folder, exist_ok=True)

    if not os.listdir(images_folder):  # If images folder is empty
        for tag in image_tags:
            image_url = f"https://source.unsplash.com/featured/?{tag}&client_id={access_key}"
            response = requests.get(image_url)
            if response.status_code == 200:
                with open(os.path.join(images_folder, f"{tag}.jpg"), 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Failed to download image for tag: {tag}")

    return images_folder

def generate_video(audio_file_name, output_video_path, images_folder):
    """Generate a video using MoviePy."""
    clips = []
    for img_file in os.listdir(images_folder):
        img_path = os.path.join(images_folder, img_file)
        clip = ImageClip(img_path).set_duration(2)  # Set duration per image
        clips.append(clip)

    if not clips:
        raise ValueError("No images found to create a video.")

    video = concatenate_videoclips(clips, method="compose")
    
    # Set FPS
    fps = 24  # Set the desired frames per second
    video.fps = fps

    audio = AudioFileClip(audio_file_name)
    video = video.set_audio(audio)

    video.write_videofile(output_video_path, codec='libx264', audio_codec='aac', fps=fps)

def generate_thumbnail(thumbnail_text, resources_path, topic_language):
    """Generate a thumbnail image."""
    from PIL import Image, ImageDraw, ImageFont

    thumbnail_path = os.path.join(resources_path, topic_language, "thumbnail.png")
    img = Image.new('RGB', (1280, 720), color='black')
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()
    draw.text((100, 100), thumbnail_text, fill="white", font=font)
    img.save(thumbnail_path)

def main():
    json_path = "video_gen.json"
    resources_path = "resources"

    # Load JSON data
    json_data = load_json_data(json_path)

    # Prepare variables and file names
    topic_language, title, description, story_narration, tags, narration_type, privacy_status, thumbnail_text, image_tags, music_suggestions, audio_language, script_revision_note, upload_date_time = prepare_variables(json_data)
    video_file_name, audio_file_name, output_video_path = prepare_file_names(topic_language, resources_path)

    # Download images
    images_folder = download_images(image_tags, resources_path, topic_language)

    # Generate audio narration
    tts = gTTS(text=story_narration['Introduction'] + ' ' + story_narration['Content'] + ' ' + story_narration['Conclusion'], lang=audio_language)
    tts.save(audio_file_name)

    # Generate video
    generate_video(audio_file_name, output_video_path, images_folder)

    # Generate thumbnail
    generate_thumbnail(thumbnail_text, resources_path, topic_language)

if __name__ == "__main__":
    main()


Moviepy - Building video resources\videos\Inception English Movie Story - English.mp4.
MoviePy - Writing audio in Inception English Movie Story - EnglishTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video resources\videos\Inception English Movie Story - English.mp4



Moviepy - Done !
Moviepy - video ready resources\videos\Inception English Movie Story - English.mp4
